* Initialize the Random Parameters
* Calculate the Predictions
* Calculate the Loss
* Calculate the Gradients
* Update the Weights
* Go to Step Two and Repeat the Process
* Stop When the Model is Good Enough

### Imports

In [23]:
from fastai.vision.all import *
from fastbook import *
from fastai.vision.widgets import *
import seaborn as sns


sns.set()


### Create Path object

In [24]:
path = untar_data(URLs.MNIST_SAMPLE)
print(path)


C:\Users\Vadim\.fastai\data\mnist_sample


### Fill the arrays of path

In [36]:
threes = (path/'train'/'3').ls().sorted()
sevens = (path/'train'/'7').ls().sorted()
print(threes[2:6])
print(f"{len(threes) = }, {len(sevens) = }")


[Path('C:/Users/Vadim/.fastai/data/mnist_sample/train/3/10011.png'), Path('C:/Users/Vadim/.fastai/data/mnist_sample/train/3/10031.png'), Path('C:/Users/Vadim/.fastai/data/mnist_sample/train/3/10034.png'), Path('C:/Users/Vadim/.fastai/data/mnist_sample/train/3/10042.png')]
len(threes) = 6131, len(sevens) = 6265


### Drawing an image 

In [26]:
Image.open(threes[0])


### Transform to tensor

In [33]:
tensor(Image.open(threes[0]))
_[3:20, 10:25]


tensor([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,  42, 118, 219, 166, 118, 118,   6,   0,   0,   0,   0,   0,   0,   0],
        [103, 242, 254, 254, 254, 254, 254,  66,   0,   0,   0,   0,   0,   0,   0],
        [ 18, 232, 254, 254, 254, 254, 254, 238,  70,   0,   0,   0,   0,   0,   0],
        [  0, 104, 244, 254, 224, 254, 254, 254, 141,   0,   0,   0,   0,   0,   0],
        [  0,   0, 207, 254, 210, 254, 254, 254,  34,   0,   0,   0,   0,   0,   0],
        [  0,   0,  84, 206, 254, 254, 254, 254,  41,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,  24, 209, 254, 254, 254, 171,   0,   0,   0,   0,   0,   0],
        [  0,   0,  91, 137, 253, 254, 254, 254, 112,   0,   0,   0,   0,   0,   0],
        [ 40, 214, 250, 254, 254, 254, 254, 254,  34,   0,   0,   0,   0,   0,   0],
        [ 81, 247, 254, 254, 254, 254, 254, 254, 146,   0,   0,   0,   0,   0,   0],
        [  0, 110, 246, 254, 254, 254, 254, 254, 171,   0,   0,  

### Create a list of tensors

In [34]:
seven_tensors = [tensor(Image.open(o)) for o in sevens]
three_tensors = [tensor(Image.open(o)) for o in threes]
len(three_tensors), len(seven_tensors)


(6131, 6265)

### Create a tensor of tensors

In [39]:
ten1 = torch.tensor([1, 2, 3])
ten2 = torch.tensor([7, 8, 9])
torch.stack([ten1, ten2])


tensor([[1, 2, 3],
        [7, 8, 9]])

6131 tensors 28*28, total: 6131 images

In [50]:
stacked_sevens = torch.stack(seven_tensors).float()/255
stacked_threes = torch.stack(three_tensors).float()/255
print(stacked_threes.shape)
# 2 image: first and second, 15 * 10
print(stacked_threes[1:3, :15, 7:17])


torch.Size([6131, 28, 28])
tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.1137, 0.5882, 0.7647, 0.9961, 1.0000, 0.9961, 0.6902, 0.7569],
         [0.1882, 0.6510, 0.8784, 0.9922, 0.9922, 0.9176, 0.7686, 0.9922, 0.9922, 0.9922],
         [0.9765, 0.9922, 0.7333, 0.1804, 0.0392, 0.0314, 0.0157, 0.0392, 0.7608, 0.9922],
         [0.9922, 0.9020, 0.1882, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.7529, 0.9922],
         [0.0784, 0.0588, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1686, 0.8784, 0.9922],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.000

### Concatenate two tensors of tensors

In [57]:
tensor_1_1 = torch.tensor([1, 2, 3])
tensor_1_2 = torch.tensor([4, 5, 6])
tensor_1 = torch.stack([ten1, ten2])

tensor_2_1 = torch.tensor([90, 80, 70])
tensor_2_2 = torch.tensor([60, 50, 40])
tensor_2 = torch.stack([ten1, ten2])

tensor_3 = torch.cat([tensor_1, tensor_2])
print(tensor_3)


tensor([[1, 2, 3],
        [7, 8, 9],
        [1, 2, 3],
        [7, 8, 9]])


view is change tensor shape

In [51]:
train_x = torch.cat([stacked_threes, stacked_sevens]).view(-1, 28 * 28)
train_x.shape


torch.Size([12396, 784])

### Create a tensor of labels for our data

In [73]:
x = torch.tensor([1, 2, 3, 4])
print(x)
print(torch.unsqueeze(x, 0))
print(torch.unsqueeze(x, 1))


tensor([1, 2, 3, 4])
tensor([[1, 2, 3, 4]])
tensor([[1],
        [2],
        [3],
        [4]])


In [62]:
train_y = tensor([1] * len(threes) + [0] * len(sevens)).unsqueeze(1)


tensor([[1],
        [1],
        [1],
        ...,
        [0],
        [0],
        [0]])


In [74]:
train_x.shape, train_y.shape


(torch.Size([12396, 784]), torch.Size([12396, 1]))

### Incoporete data and targets to dataset

The list of tensors ```[[data1, answer1], ...]```

In [75]:
dset = list(zip(train_x, train_y))
x, y = dset[0]
x.shape, y


(torch.Size([784]), tensor([1]))

### Create a DataLoader object with batch_size = 256

In [76]:
dl = DataLoader(dset, batch_size=256)
xb, yb = first(dl)
xb.shape, yb.shape


(torch.Size([256, 784]), torch.Size([256, 1]))

### Do the same with the valid data

In [77]:
valid_3_tens = torch.stack([tensor(Image.open(o))
                           for o in (path/'valid'/'3').ls()])
valid_3_tens = valid_3_tens.float()/255

valid_7_tens = torch.stack([tensor(Image.open(o))
                           for o in (path/'valid'/'7').ls()])
valid_7_tens = valid_7_tens.float()/255

print(valid_3_tens.shape, valid_7_tens.shape)

valid_x = torch.cat([valid_3_tens, valid_7_tens]).view(-1, 28 * 28)
valid_y = tensor([1] * len(valid_3_tens) + [0] *
                 len(valid_7_tens)).unsqueeze(1)

valid_dset = list(zip(valid_x, valid_y))
valid_dl = DataLoader(valid_dset, batch_size=256)


torch.Size([1010, 28, 28]) torch.Size([1028, 28, 28])


### Set a learning rate 

In [79]:
lr = 1.0


### Define functions

```python
where(condition, case_true, case_false)
if condition: case_true
else: case_false
```

func mnist_loss always return loss between [0, 1]

In [80]:
def mnist_loss(predictions, targets):
    return torch.where(targets == 1, 1 - predictions, predictions).mean()


model -- is a function that calculates predictions

In [81]:
def calc_grad(xb, yb, model):
    preds = model(xb)
    loss = mnist_loss(preds, yb)
    loss.backward()


In [82]:
def batch_accuracy(xb, yb):
    preds = xb.sigmoid()
    correct = (preds > 0.5) == yb
    return correct.float().mean()


### Optimizer

nn.Linear does the same thing as our init_params and linear together. It
contains both the weights and biases in a single class

In [84]:
linear_model = nn.Linear(28*28, 1)


Every PyTorch module knows what parameters it has that can be trained; they are
available through the parameters method:

In [85]:
w, b = linear_model.parameters()
w.shape, b.shape


(torch.Size([1, 784]), torch.Size([1]))

### Create own optimizer

In [86]:
class BasicOptim:
    def __init__(self, params, lr):
        self.params = list(params)
        self.lr = lr

    def step(self):
        for p in self.params:
            p.data -= p.grad.data * self.lr

    def zero_grad(self):
        for p in self.params:
            p.grad = None


Create BasicOptim objects with defined parametrs

In [87]:
opt = BasicOptim(linear_model.parameters(), lr)


In [88]:
def train_epoch(model):
    for x_batch, y_batch in dl:
        calc_grad(x_batch, y_batch, model)
        opt.step()
        opt.zero_grad()


In [89]:
def validate_epoch(model):
    accs = [batch_accuracy(model(xb), yb) for xb, yb in valid_dl]
    return round(torch.stack(accs).mean().item(), 4)


In [90]:
validate_epoch(linear_model)


0.4705

### One train loop (20 epoches)

In [91]:
def train_model(model, epochs):
    for _ in range(epochs):
        train_epoch(model)
        print(validate_epoch(model), end=' ')


In [92]:
train_model(linear_model, 20)


0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 

### Creater optimiser using fastai

In [93]:
linear_model = nn.Linear(28*28, 1)
opt = SGD(linear_model.parameters(), lr)
train_model(linear_model, 20)


0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 0.9535 

fastai also provides Learner.fit, which we can use instead of train_model.

To create a Learner, we first need to create a DataLoaders, by passing in our
training and validation DataLoaders

In [94]:
dls = DataLoaders(dl, valid_dl)


In [96]:
learn = Learner(dls,
                nn.Linear(28*28, 1),
                opt_func=SGD,
                loss_func=mnist_loss,
                metrics=batch_accuracy)


In [97]:
learn.fit(n_epoch=10, lr=lr)


epoch,train_loss,valid_loss,batch_accuracy,time
0,-83.022614,-335.850677,0.953386,00:00
1,-309.395782,-672.167480,0.953386,00:00
2,-570.783752,-1008.484375,0.953386,00:00
3,-855.037354,-1344.801270,0.953386,00:00
4,-1152.467773,-1681.118042,0.953386,00:00
5,-1456.801025,-2017.434937,0.953386,00:00
6,-1764.505493,-2353.751953,0.953386,00:00
7,-2073.773682,-2690.068604,0.953386,00:00
8,-2383.741943,-3026.385742,0.953386,00:00
9,-2694.014160,-3362.702148,0.953386,00:00
